In [ ]:
import re
import math
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import json
import numpy as np
import logging
from sentence_transformers import SentenceTransformer, util

In [ ]:
# Setup logging
logging.basicConfig(level=logging.INFO)

In [ ]:
def gold_path(instance, json_file):
    # Load the JSON file
    with open(json_file, 'r') as file:
        try:
            data = json.load(file)
            routes = data if isinstance(data, list) else [data]
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(routes)

    # Specify the instance ID you want to look for
    target_instance_id = str(instance)  # Replace with the actual instance ID
    data = []
    data.insert(0, "Starting Point")
    # Explore every instance in the 'instances' column
    for index, row in df.iterrows():
        instances = row['instances']  # Get the instances dictionary

        if isinstance(instances, dict):
            if target_instance_id in instances:
                instance_data = instances[target_instance_id]

                # Access the gold navigation lines
                gold_navigation_lines = instance_data.get('gold_navigation_lines', [])

                # Ensure gold_navigation_lines is a list
                if isinstance(gold_navigation_lines, list):
                    # Initialize a list to store output lines
                    output_lines = []

                    # Loop through the navigation lines
                    for i, line in enumerate(gold_navigation_lines):
                        # Skip lines containing "stop"
                        if "stop" in line.strip().lower():
                            line=""
                            break

                        # Replace lines containing 'intersection' with 'Intersection'
                        if 'intersection' in line.lower():
                            line = 'Intersection'

                        # Print the first line
                        if i == 0:
                            output_lines.append(line)  # Print the first line

                        # Check if the line does not start with '- n.' where n is a number
                        if not re.match(r'^\s*\d+\.', line):
                            output_lines.append(line)  # Add the line without the match

                            # Check if there's a next line and add it
                            if i + 1 < len(gold_navigation_lines):
                                output_lines.append(gold_navigation_lines[i + 1])  # Add the next line

                    # Print the selected lines
                    for line in output_lines:
                        data.append(line)  # Collect output lines in data

                else:
                    print(f"Instance ID {target_instance_id} does not contain valid navigation lines.")
                break  # Exit after finding the target instance
        else:
            print(f"Row {index} does not contain a valid instances dictionary.")
    #data.insert(-1, "Target")

    # Initialize lists to hold places and actions
    places = []
    actions = []

    # Iterate through the data and classify each entry
    for entry in data:
        if re.match(r'^\d+\.\s*(forward|right|left|stop)', entry):  # Match actions
            actions.append(entry.split('. ')[1])  # Add only the action (no number)
        else:
            # Replace unwanted phrases and clean the entry
            cleaned_entry = re.sub(r' slightly right\.| ahead\.| slightly left\.| on your left\.| on your right\.', '', entry)
            # Check if the cleaned entry has at most 2 words
            if len(cleaned_entry.split()) <= 4:
                places.append(cleaned_entry)  # Add cleaned entry to places

    # Remove duplicates while retaining the first occurrence and ensuring "Intersection" is included
    unique_places = []
    unique_actions = []

    for i in range(len(places)):
        place = places[i]
        action = actions[i] if i < len(actions) else None

        # Check if the place is "Intersection" or not already in unique_places
        if place == "Intersection":
            unique_places.append(place)

        elif place not in unique_places:  # Corrected condition
            unique_places.append(place)

    # Output the separated lists
    return unique_places

    # Example usage
json_file_path = "/content/dev_results.json"  # Replace with your file path
instance_id_to_search = "6918"
node_gold =  gold_path(instance_id_to_search, json_file_path)

# Print the combined lines
for line in node_gold:
    print(line)



Starting Point
Project Cozy
Intersection
Intersection
Intersection
Intersection
Intersection


In [ ]:
import json
import pandas as pd

def predicted_path(predict_instance, predicted_file):

    # Load the JSON file
    with open(predicted_file, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            return [], []

    # Convert JSON data to DataFrame
    df = pd.DataFrame(data)

    # Search for the matching instance ID
    matching_row = df[df["Instance_id"] == int(predict_instance)]

    if matching_row.empty:
        print(f"No matching instance found for ID: {predict_instance}")
        return [], []

    # Extract the graph data
    graph_list = matching_row.iloc[0]["Graph"]

    # Initialize lists for nodes and actions
    nodes = []
    my_actions = []

    if isinstance(graph_list, dict):
        nodes = [node.title() for node in graph_list.get('Nodes', {}).values()]
        edges = graph_list.get('Edges', [])
        my_actions = [edge.get('action', '').lower() for edge in edges]

    # Define valid actions in lowercase
    valid_actions = ["forward", "left", "right", "stop"]

    # Replace invalid actions with "forward"
    my_actions = [action if action in valid_actions else "forward" for action in my_actions]

    # Clean the nodes
    cleaned_nodes = [
        "Intersection" if any(key.lower() in node.lower() for key in ["intersection", "light", "4-way stop", "t-intersection", "lights", "junction"])
        #else "Target" if any(key.lower() in node.lower() for key in ["target", "(target)"])
        else node.title()
        for node in nodes
    ]
    new_nodes = []
    for my_node in cleaned_nodes:
        if "Target" in my_node:  # Check if "Target" is in the node
            my_node = my_node.replace("Target", "").replace("()", "").strip()  # Remove "(Target)" and clean spaces
            # Add the modified node only if it's not empty
            if my_node != "":
                new_nodes.append(my_node)  # Add the modified node
        else:
            new_nodes.append(my_node)  # Add the node as is

    # Remove the last node if it exists
    #if new_nodes:
     #   new_nodes.pop()
    cleaned_nodes = new_nodes


    return cleaned_nodes

# Path to your JSON file
json_file_predict = "/content/cleaned_cognitive_map_cost.json"

# Specify the instance ID to search for
instance_id_to_search = "6918"

# Get predicted nodes and actions
node_predict = predicted_path(instance_id_to_search, json_file_predict)

# Print the cleaned nodes and actions
print("\nNodes:")
for line_node in node_predict:
    print(line_node)


Nodes:
Starting Point, Parking
Project Cozy
Intersection
Intersection
Intersection
Intersection
Intersection
Bagel Shop
Intersection


In [ ]:
# Path to your JSON file
json_file_predict = '/content/cleaned_cognitive_map_cost.json'
# Path to your JSON file
json_file_GT = '/content/dev_results.json'

# Load the JSON file
with open(json_file_predict, 'r') as file:
    try:
        # Load the entire JSON content
        data = json.load(file)
        # Ensure the data is a list of dictionaries
        routes = data if isinstance(data, list) else [data]
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON file: {e}")
        routes = []

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(routes)

# Initialize dictionaries
all_predicted_nodes = {}
all_gold_nodes = {}

# Loop over each graph instance
for i in range(0, 200):
    instance_id = df.loc[i, 'Instance_id']

    # Get Ground Truth nodes
    node_GT = gold_path(instance_id, json_file_GT)

    # Get predicted nodes
    node_predict = predicted_path(instance_id, json_file_predict)

    # Store results in dictionaries
    all_predicted_nodes[instance_id] = node_predict
    all_gold_nodes[instance_id] = node_GT

In [ ]:
# Print or process results
print("Predicted Nodes:", all_predicted_nodes)
print("Gold Nodes:", all_gold_nodes)

Predicted Nodes: {np.int64(6918): ['Starting Point, Parking', 'Project Cozy', 'Intersection', 'Intersection', 'Intersection', 'Intersection', 'Intersection', 'Bagel Shop', 'Intersection'], np.int64(990): ['Starting Point, Bike Rental', 'Intersection', 'Intersection', 'Intersection', 'Intersection', "Dunkin' Donuts", 'Intersection'], np.int64(1017): ['Intersection', 'Intersection', 'Intersection', 'Intersection', 'Intersection', 'Lin Zexu', 'Bike Rental', 'Chatham Green Professional Suites', 'Intersection', 'St. James Church,'], np.int64(5947): ['Intersection', 'Overpass,', 'Intersection', 'Intersection', 'Bike Rental', 'Key Food'], np.int64(6503): ['Starting Point, Outdoor Dining', 'Intersection', 'Vbar Seaport', 'Intersection', 'Abercrombie', 'Intersection', 'Intersection', 'Intersection', 'Intersection', "Mcdonald'S", 'Florist', 'Intersection'], np.int64(1681): ['Intersection', 'Paulaner Restaurant', 'Intersection', 'Bt Kitchen Supplies Inc.', 'Loreley Beer Garden,  For Lease Sign, T

Task Completion Rate (TC)

In [ ]:
def compute_task_completion_rate(predicted_paths, gold_paths):
    """
    Computes the Task Completion Rate (TC) for each instance and prints results.

    Parameters:
    - predicted_paths (dict): Dictionary {instance_id: [list of predicted nodes]}.
    - gold_paths (dict): Dictionary {instance_id: [list of ground truth nodes]}.

    Returns:
    - float: Average Task Completion Rate (TC) as a percentage.
    """
    successful_tasks = 0
    total_tasks = len(gold_paths)

    if total_tasks == 0:
        return 0.0  # Avoid division by zero

    print("\nTask Completion Rate per Sample:")
    print("-" * 50)

    for instance_id, gold_path in gold_paths.items():
        predicted_path = predicted_paths.get(instance_id, [])

        if not predicted_path or not gold_path:
            print(f"Instance {instance_id}: No valid paths found.")
            continue  # Skip if either path is missing

        # Get the predicted final node and second-to-last predicted node (if available)
        predicted_final_node = predicted_path[-1]

        target_node = gold_path[-1]

        # Allow success if the final predicted node or second-to-last node is the target or its immediate neighbor
        neighboring_nodes = set(gold_path[-2:]) if len(gold_path) > 1 else {target_node}

        success = predicted_final_node == target_node or predicted_final_node in neighboring_nodes

        if success:
            successful_tasks += 1

        print(f"Instance {instance_id}: {'1 Success' if success else '0 Failure'}")

    avg_tc = successful_tasks / total_tasks  # 0.0 - 1.0
    print("\n" + "=" * 50)
    print(f"Overall Task Completion Rate (TC): {avg_tc:.2%}")

    return avg_tc * 100

# Example Usage
predicted_paths = all_predicted_nodes
gold_paths = all_gold_nodes

tc = compute_task_completion_rate(predicted_paths, gold_paths)



Task Completion Rate per Sample:
--------------------------------------------------
Instance 6918: 1 Success
Instance 990: 1 Success
Instance 1017: 0 Failure
Instance 5947: 1 Success
Instance 6503: 1 Success
Instance 1681: 0 Failure
Instance 5327: 0 Failure
Instance 6960: 0 Failure
Instance 2663: 0 Failure
Instance 6696: 1 Success
Instance 993: 0 Failure
Instance 1994: 1 Success
Instance 4552: 1 Success
Instance 2840: 1 Success
Instance 96: 1 Success
Instance 1966: 0 Failure
Instance 6825: 1 Success
Instance 5180: 0 Failure
Instance 5807: 0 Failure
Instance 2879: 0 Failure
Instance 3513: 0 Failure
Instance 4806: 1 Success
Instance 4693: 1 Success
Instance 2523: 1 Success
Instance 5087: 1 Success
Instance 5856: 1 Success
Instance 6700: 0 Failure
Instance 2946: 1 Success
Instance 1921: 1 Success
Instance 1939: 1 Success
Instance 3997: 1 Success
Instance 5523: 1 Success
Instance 5197: 1 Success
Instance 1570: 1 Success
Instance 2045: 1 Success
Instance 3062: 0 Failure
Instance 1047: 0 Fa

Point Accuracy (KPA)

In [ ]:
# Load a pre-trained SBERT model
model = SentenceTransformer("all-MiniLM-L6-v2")

def compute_kpa_semantic(predicted_keypoints, ground_truth_keypoints, similarity_threshold=0.7):
    """
    Computes Key Point Accuracy (KPA) using semantic similarity.
    :param predicted_keypoints: List of predicted keypoints (nodes).
    :param ground_truth_keypoints: List of ground truth keypoints (nodes).
    :param similarity_threshold: Cosine similarity threshold for a match.
    :return: KPA score (float).
    """
    if not predicted_keypoints or not ground_truth_keypoints:
        return 0.0  # Avoid division by zero

    correct_matches = 0

    # Encode keypoints into embeddings
    pred_embeddings = model.encode(predicted_keypoints, convert_to_tensor=True)
    gt_embeddings = model.encode(ground_truth_keypoints, convert_to_tensor=True)

    # Compute cosine similarity matrix
    similarity_matrix = util.pytorch_cos_sim(pred_embeddings, gt_embeddings)

    # Check for matches using the threshold
    for i in range(len(predicted_keypoints)):
        max_sim = similarity_matrix[i].max().item()
        if max_sim >= similarity_threshold:
            correct_matches += 1

    # Compute KPA
    return correct_matches / len(ground_truth_keypoints)

# Example Data
# Compute KPA for each instance
kpa_results = {}
prom_kpa = 0

for instance_id in all_predicted_nodes.keys():
    kpa = compute_kpa_semantic(all_predicted_nodes[instance_id], all_gold_nodes[instance_id])
    kpa_results[instance_id] = kpa
    prom_kpa += kpa
    print(f"Instance {instance_id}: KPA = {kpa:.2f}")

# Compute Average KPA
total_kpa = prom_kpa / len(kpa_results)
print(f"Average KPA = {total_kpa:.2f}")

Instance 6918: KPA = 1.00
Instance 990: KPA = 1.00
Instance 1017: KPA = 0.82
Instance 5947: KPA = 0.71
Instance 6503: KPA = 0.90
Instance 1681: KPA = 0.43
Instance 5327: KPA = 0.88
Instance 6960: KPA = 1.00
Instance 2663: KPA = 0.75
Instance 6696: KPA = 1.00
Instance 993: KPA = 0.67
Instance 1994: KPA = 1.14
Instance 4552: KPA = 1.00
Instance 2840: KPA = 0.70
Instance 96: KPA = 0.86
Instance 1966: KPA = 1.00
Instance 6825: KPA = 0.88
Instance 5180: KPA = 0.83
Instance 5807: KPA = 0.70
Instance 2879: KPA = 1.00
Instance 3513: KPA = 0.86
Instance 4806: KPA = 1.20
Instance 4693: KPA = 0.88
Instance 2523: KPA = 0.82
Instance 5087: KPA = 0.86
Instance 5856: KPA = 1.14
Instance 6700: KPA = 0.71
Instance 2946: KPA = 1.00
Instance 1921: KPA = 1.00
Instance 1939: KPA = 1.12
Instance 3997: KPA = 1.14
Instance 5523: KPA = 1.00
Instance 5197: KPA = 1.00
Instance 1570: KPA = 1.00
Instance 2045: KPA = 1.00
Instance 3062: KPA = 0.78
Instance 1047: KPA = 0.83
Instance 5783: KPA = 1.00
Instance 3279: K

In [ ]:
def gold_path(instance, json_file):
    # Load the JSON file
    with open(json_file, 'r') as file:
        try:
            data = json.load(file)
            routes = data if isinstance(data, list) else [data]
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(routes)

    # Specify the instance ID you want to look for
    target_instance_id = str(instance)  # Replace with the actual instance ID
    data = []
    data.insert(0, "Starting Point")
    # Explore every instance in the 'instances' column
    for index, row in df.iterrows():
        instances = row['instances']  # Get the instances dictionary

        if isinstance(instances, dict):
            if target_instance_id in instances:
                instance_data = instances[target_instance_id]

                # Access the gold navigation lines
                gold_navigation_lines = instance_data.get('gold_navigation_lines', [])

                # Ensure gold_navigation_lines is a list
                if isinstance(gold_navigation_lines, list):
                    # Initialize a list to store output lines
                    output_lines = []

                    # Loop through the navigation lines
                    for i, line in enumerate(gold_navigation_lines):
                        # Skip lines containing "stop"
                        if "stop" in line.strip().lower():
                            line=""
                            break

                        # Replace lines containing 'intersection' with 'Intersection'
                        #if 'intersection' in line.lower():
                         #   line = 'Intersection'

                        # Print the first line
                        if i == 0:
                            output_lines.append(line)  # Print the first line

                        # Check if the line does not start with '- n.' where n is a number
                        if not re.match(r'^\s*\d+\.', line):
                            output_lines.append(line)  # Add the line without the match

                            # Check if there's a next line and add it
                            if i + 1 < len(gold_navigation_lines):
                                output_lines.append(gold_navigation_lines[i + 1])  # Add the next line

                    # Print the selected lines
                    for line in output_lines:
                        data.append(line)  # Collect output lines in data
                    #print(data)
                else:
                    print(f"Instance ID {target_instance_id} does not contain valid navigation lines.")
                break  # Exit after finding the target instance
        else:
            print(f"Row {index} does not contain a valid instances dictionary.")
    data.insert(-1, "Target")

    # Initialize lists to hold places and actions
    places = []
    actions = []

    # Iterate through the data and classify each entry
    for entry in data:
        if re.match(r'^\d+\.\s*(forward|right|left|stop)', entry):  # Match actions
            actions.append(entry.split('. ')[1])  # Add only the action (no number)
        else:
            # Replace unwanted phrases and clean the entry
            cleaned_entry = re.sub(r' slightly right\.| ahead\.| slightly left\.|There is a | on your left\.| on your right\.', '', entry)
            # Check if the cleaned entry has at most 2 words
            if len(cleaned_entry.split()) <= 5:
                places.append(cleaned_entry)  # Add cleaned entry to places

    # Remove duplicates while retaining the first occurrence and ensuring "Intersection" is included
    unique_places = []
    unique_actions = []

    for i in range(len(places)):
        place = places[i]
        action = actions[i] if i < len(actions) else None


        # Check if the place is "Intersection" or not already in unique_places
        if "Intersection" in place or "intersection" in place:
            unique_places.append(place)
            if action:
              unique_actions.append(action)
        elif place not in unique_places:  # Corrected condition
            unique_places.append(place)
            if action:
              unique_actions.append(action)
    # Output the separated lists
    return unique_places#, unique_actions

    # Example usage
json_file_path = "/content/dev_results.json"  # Replace with your file path
instance_id_to_search = "6918"
node_gold =  gold_path(instance_id_to_search, json_file_path)

# Print the combined lines
for line in node_gold:
    print(line)

Starting Point
Project Cozy
3-way intersection. Project Cozy
4-way intersection.
4-way intersection.
4-way intersection.
4-way intersection.
Target


In [ ]:
import json
import pandas as pd

def predicted_path(predict_instance, predicted_file):

    # Load the JSON file
    with open(predicted_file, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            return [], []

    # Convert JSON data to DataFrame
    df = pd.DataFrame(data)

    # Search for the matching instance ID
    matching_row = df[df["Instance_id"] == int(predict_instance)]

    if matching_row.empty:
        print(f"No matching instance found for ID: {predict_instance}")
        return [], []

    # Extract the graph data
    graph_list = matching_row.iloc[0]["Graph"]

    # Initialize lists for nodes and actions
    nodes = []
    my_actions = []

    if isinstance(graph_list, dict):
        nodes = [node.title() for node in graph_list.get('Nodes', {}).values()]
        edges = graph_list.get('Edges', [])
        my_actions = [edge.get('action', '').lower() for edge in edges]

    # Define valid actions in lowercase
    valid_actions = ["forward", "left", "right", "stop"]

    # Replace invalid actions with "forward"
    my_actions = [action if action in valid_actions else "forward" for action in my_actions]

    # Clean the nodes
    cleaned_nodes = [
        "Target" if any(key.lower() in node.lower() for key in ["target", "(target)"])
        #else "Intersection" if any(key.lower() in node.lower() for key in ["intersection", "light", "4-way stop", "t-intersection", "lights", "junction"])
        else node.replace("T-Intersection", "3-Way Intersection").strip().title()
        for node in nodes
    ]
    new_nodes = []
    for my_node in cleaned_nodes:
        if "Target" in my_node:  # Check if "Target" is in the node
            my_node = my_node.replace("()", "").strip()  # Remove "(Target)" and clean spaces
            # Add the modified node only if it's not empty
            if my_node != "":
                new_nodes.append(my_node)  # Add the modified node
        else:
            new_nodes.append(my_node)  # Add the node as is

    # Remove the last node if it exists
    #if new_nodes:
     #   new_nodes.pop()
    #cleaned_nodes = new_nodes


    return cleaned_nodes#, my_actions

# Path to your JSON file
json_file_predict = "/content/cleaned_cognitive_map_cost.json"

# Specify the instance ID to search for
instance_id_to_search = "6918"

# Get predicted nodes and actions
node_predict = predicted_path(instance_id_to_search, json_file_predict)

# Print the cleaned nodes and actions
print("\nNodes:")
for line_node in node_predict:
    print(line_node)


Nodes:
Starting Point, Parking
Project Cozy
Intersection, Parking
3-Way Intersection, Mille Miglia
Intersection
4-Way Intersection, Laundromat
4-Way Intersection
Bagel Shop
Target


In [ ]:
# Path to your JSON file
json_file_predict = '/content/cleaned_cognitive_map_cost.json'
# Path to your JSON file
json_file_GT = '/content/dev_results.json'

# Load the JSON file
with open(json_file_predict, 'r') as file:
    try:
        # Load the entire JSON content
        data = json.load(file)
        # Ensure the data is a list of dictionaries
        routes = data if isinstance(data, list) else [data]
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON file: {e}")
        routes = []

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(routes)

# Initialize dictionaries
all_predicted_nodes = {}
all_gold_nodes = {}

# Loop over each graph instance
for i in range(0, 200):
    instance_id = df.loc[i, 'Instance_id']

    # Get Ground Truth nodes
    node_GT = gold_path(instance_id, json_file_GT)

    # Get predicted nodes
    node_predict = predicted_path(instance_id, json_file_predict)

    # Store results in dictionaries
    all_predicted_nodes[instance_id] = node_predict
    all_gold_nodes[instance_id] = node_GT

Shortest Path Distance (SPD)

In [ ]:
def find_target_index(path, target_label='Target'):
    """
    Find the index of a node containing the word 'Target' in the path.
    Returns -1 if not found.
    """
    for idx, node in enumerate(path):
        # If node is a list (multiple labels), join it into a string
        node_str = ' '.join(node) if isinstance(node, list) else node

        # Check if target_label is in the node name (case-insensitive)
        if target_label.lower() in node_str.lower():
            return idx  # Found, return the index

    return -1  # Not found

def compare_target_index(predicted_path, gold_path):
    """
    Compare the index of the target node in predicted path to the length of the gold path.
    Returns a dictionary with target indexes, index difference, and progress ratio.
    """
    # Search for target indices
    target_index_predicted = find_target_index(predicted_path)
    target_index_gold = abs(find_target_index(gold_path))

    # Initialize with default values
    index_difference = float('inf')
    progress_ratio = 0.0

    # Handle case where target is found in the predicted path
    if target_index_predicted == -1:
      target_index_predicted = len(predicted_path)
    if target_index_predicted != -1:
        index_difference = abs(target_index_predicted - target_index_gold)

        # Avoid division by zero
        if len(predicted_path) > 0:
            ratio_spd = abs(index_difference/len(predicted_path))

    # Print diagnostic info
    print(f"Predicted Target Index: {target_index_predicted}")
    print(f"Gold Target Index: {target_index_gold}")
    print(f"Index Difference (SPD proxy): {index_difference}")
    print(f"Ratio SPD: {ratio_spd:.2f}")

    return {
        'target_index_predicted': target_index_predicted,
        'target_index_gold': target_index_gold,
        'index_difference': index_difference,
        'ratio_spd': ratio_spd
    }


# Store all SPD-like index differences
spd_scores = {}

# Loop through all instances in your data
for instance_id in all_predicted_nodes.keys():
    predicted_path = all_predicted_nodes[instance_id]
    gold_path = all_gold_nodes[instance_id]

    # Run the target index comparison for each instance
    comparison = compare_target_index(predicted_path, gold_path)

    # Grab the SPD-like index difference
    spd = comparison['ratio_spd']

    # Store it in the dictionary with the instance_id
    spd_scores[instance_id] = spd

    print(f"Instance {instance_id}: SPD (index difference) = {spd}")

# Calculate average SPD
if spd_scores:
    total_spd = sum(spd_scores.values())
    avg_spd = total_spd / len(spd_scores)
    print(f"\nAverage SPD (index difference) over {len(spd_scores)} instances: {avg_spd:.2f}")
else:
    print("No SPD scores calculated.")

Predicted Target Index: 8
Gold Target Index: 7
Index Difference (SPD proxy): 1
Ratio SPD: 0.11
Instance 6918: SPD (index difference) = 0.1111111111111111
Predicted Target Index: 6
Gold Target Index: 6
Index Difference (SPD proxy): 0
Ratio SPD: 0.00
Instance 990: SPD (index difference) = 0.0
Predicted Target Index: 9
Gold Target Index: 12
Index Difference (SPD proxy): 3
Ratio SPD: 0.30
Instance 1017: SPD (index difference) = 0.3
Predicted Target Index: 6
Gold Target Index: 7
Index Difference (SPD proxy): 1
Ratio SPD: 0.14
Instance 5947: SPD (index difference) = 0.14285714285714285
Predicted Target Index: 11
Gold Target Index: 10
Index Difference (SPD proxy): 1
Ratio SPD: 0.08
Instance 6503: SPD (index difference) = 0.08333333333333333
Predicted Target Index: 4
Gold Target Index: 7
Index Difference (SPD proxy): 3
Ratio SPD: 0.60
Instance 1681: SPD (index difference) = 0.6
Predicted Target Index: 8
Gold Target Index: 8
Index Difference (SPD proxy): 0
Ratio SPD: 0.00
Instance 5327: SPD (in